In [1]:
# trying to install on 3.11 fails trying to build wheel files; see https://github.com/yaml/pyyaml/issues/724
!rye add pyyaml==5.3 mlrun>=1.6.4  scikit-learn~=1.4.0  numpy~=1.26 protobuf==3.19.6


Building file:///home/bcorfman/dev/mlrun-trial             ░░░░░░░░░░░░░░░░░░░░ [0/0] Fetching packages...                                 
   Built file:///home/bcorfman/dev/mlrun-trial    
   Built file:///home/bcorfman/dev/mlrun-trial    
   Built file:///home/bcorfman/dev/mlrun-trialv/ml
   Built file:///home/bcorfman/dev/mlrun-trial                                  Built 1 editable in 440ms
Uninstalled 1 package in 3ms
Installed 1 package in 1ms==0.1.0 (from file:///home/bcorfman/dev/ml
 - mlrun-trial==0.1.0 (from file:///home/bcorfman/dev/mlrun-trial)
 + mlrun-trial==0.1.0 (from file:///home/bcorfman/dev/mlrun-trial)


In [2]:
import mlrun

In [3]:
project = mlrun.get_or_create_project("quick-tutorial", "./", user_project=True)

> 2024-07-02 23:14:31,264 [info] Project loaded successfully: {'project_name': 'quick-tutorial'}


In [4]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

import mlrun


@mlrun.handler(outputs=["dataset", "label_column"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["label"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset, "label"

Writing data-prep.py


In [5]:
data_gen_fn = project.set_function(
    "data-prep.py",
    name="data-prep",
    kind="job",
    image="mlrun/mlrun",
    handler="breast_cancer_generator",
)
project.save()  # save the project with the latest config